
# TestNb



## Setup (installing necessary libraries)

In [266]:
# !pip install "tensorflow-text>=2.10"
# !pip install einops

#Importing Libraries 

In [363]:
import numpy as np
import typing
from typing import Any, Tuple
from sklearn.model_selection import train_test_split

import pathlib

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

#Defining the Shapechecker

In [364]:
#@title
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

# Loading the Dataset

In [365]:
import pandas as pd
ORM_data = pd.read_excel('test_ds.xlsx')


#Reading Data from Dataset

In [366]:
ORM_data.head()

,OM_Regular,OM_Prediction
0,moduleOM_name:0openDeclarationonesi,"moduleOM_name:0openDeclarationones,P"
1,moduleOM_name:0openDeclarationonesig,"moduleOM_name:0openDeclarationonesig​,NP"
2,moduleOM_name:0openDeclarationonesigclass1_name,moduleOM_name:0openDeclarationonesigclass1_nam...
3,moduleOM_name:0openDeclarationone,"moduleOM_name:0openDeclarationone,P"
4,moduleOM_name:0openDeclaration,"moduleOM_name:0openDeclaration,NP"


In [367]:
OM_Regular = ORM_data['OM_Regular'].values
OM_Prediction = ORM_data['OM_Prediction'].values

In [368]:
X = OM_Regular
Y = OM_Prediction

#### Dividing data as Target and Context

In [369]:
target_raw =  Y
context_raw = X
print(context_raw[-1])

moduleOM_name:0openDeclaration


In [370]:
print(target_raw[-1])

moduleOM_name:0openDeclaration,NP


### Create a tf.data dataset

From these arrays of strings you can create a `tf.data.Dataset` of strings that shuffles and batches them efficiently:

In [371]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 1

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [372]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

tf.Tensor([b'moduleOM_name:0openDeclarationonesigclass1_name'], shape=(1,), dtype=string)

tf.Tensor([b'moduleOM_name:0openDeclarationonesigclass1_name,NP'], shape=(1,), dtype=string)


### Text preprocessing

One of the goals of this tutorial is to build a model that can be exported as a `tf.saved_model`. To make that exported model useful it should take `tf.string` inputs, and return `tf.string` outputs: All the text processing happens inside the model. Mainly using a `layers.TextVectorization` layer.

#### Standardization

The model is dealing with multilingual text with a limited vocabulary. So it will be important to standardize the input text.

The first step is Unicode normalization to split accented characters and replace compatibility characters with their ASCII equivalents.

The `tensorflow_text` package contains a unicode normalize operation, We may or may not decide to Use this for ORM data. I kept it in the experiment

In [373]:

example_text = tf.constant('moduleOM_name:0openDeclarationonesi')
print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

b'moduleOM_name:0openDeclarationonesi'
b'moduleOM_name:0openDeclarationonesi'


In [374]:



def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text,'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text,'', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text,'', r'')
  # Strip whitespace.
  text = tf.strings.strip(text)
 
  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text




In [375]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

moduleOM_name:0openDeclarationonesi
[START] moduleom_name:0opendeclarationonesi [END]


#### Text Vectorization

This standardization function will be wrapped up in a `tf.keras.layers.TextVectorization` layer which will handle the vocabulary extraction and conversion of input text to sequences of tokens.

In [376]:
max_vocab_size = 4000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

The `TextVectorization` layer and many other [Keras preprocessing layers](https://www.tensorflow.org/guide/keras/preprocessing_layers) have an `adapt` method. This method reads one epoch of the training data, and works a lot like `Model.fit`. This `adapt` method initializes the layer based on the data. Here it determines the vocabulary:

In [377]:
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:10]

['',
 '[UNK]',
 '[START]',
 '[END]',
 'moduleom_name:0opendeclarationonesigclass1_name',
 'moduleom_name:0opendeclarationone',
 'moduleom_name:0opendeclaration']

That's the context data  `TextVectorization` layer, now build and `.adapt()` for the Target Data one:

In [378]:
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

['',
 '[UNK]',
 '[START]',
 '[END]',
 'moduleom_name:0opendeclarationonesigclass1_name,np',
 'moduleom_name:0opendeclarationone,p',
 'moduleom_name:0opendeclaration,np']

Now these layers can convert a batch of strings into a batch of token IDs:

The `get_vocabulary` method can be used to convert token IDs back to text:

The returned token IDs are zero-padded. This can easily be turned into a mask: